### Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from keras.utils import np_utils
from config import db_password
import psycopg2
import pandas as pd
import tensorflow as tf

In [2]:
# Import and read csv
# import pandas as pd 
# kushy_washington_cleaned_df = pd.read_csv("./Data/kushy_washington_cleaned.csv")
# print(kushy_washington_cleaned_df.shape)
# kushy_washington_cleaned_df.head()

In [3]:
# Connect to sql database
con = psycopg2.connect(database='Medical-Marijuana-Group', user = 'postgres', password = db_password, host = '127.0.0.1', port = '5432')

# Create a new cursor
cur = con.cursor()

print('Opened database successfully')

Opened database successfully


In [4]:
# Create function that takes in a PostgreSQL query and outputs a Pandas dataframe
def create_pandas_df(sql_query, database = con):
    df = pd.read_sql_query(sql_query, database)
    return df
  
# Utilize the create_pandas_df function to create a Pandas dataframe
# Store the data as a variable
kushy_washington_cleaned_df = create_pandas_df('SELECT * FROM kushy_washington_cleaned')
print(kushy_washington_cleaned_df.shape)
kushy_washington_cleaned_df.head()

(722, 36)


,test_id,test_strain,type,ailment_1,ailment_2,ailment_3,ailment_4,ailment_5,effects_1,effects_2,...,inventory_type,chemotype,strain_leafly_page_rank,strain_leafly_review_rank,strain_tests,strain_chemotype,strain_prop_chemotype1,strain_prop_chemotype2,strain_prop_chemotype3,strain_modal_chemo_prop
0,19,Dutch Hawaiian,Sativa,Depression,Stress,Pain,Nausea,None,Happy,Energetic,...,Flower Lot,1,619,141,425,1,1.0,0.0,0.0,1.0
1,21,SleeStack,Hybrid,Stress,Depression,None,None,None,Uplifted,Euphoric,...,Flower Lot,1,1276,151,63,1,1.0,0.0,0.0,1.0
2,22,Space Needle,Sativa,Depression,Stress,None,None,None,Uplifted,Euphoric,...,Flower Lot,1,0,0,0,0,0.0,0.0,0.0,0.0
3,23,Double Purple Doja,Hybrid,Pain,Stress,Inflammation,Muscle Spasms,None,Relaxed,Creative,...,Flower Lot,1,1115,151,52,1,1.0,0.0,0.0,1.0
4,37,Jack Skellington,Hybrid,Stress,Depression,Pain,Muscle Spasms,None,Creative,Relaxed,...,Flower Lot,1,708,141,124,1,1.0,0.0,0.0,1.0


In [5]:
# Close the cursor and connection so the server can allocate bandwidth to other requests
cur.close()
con.close()
print('Closed database successfully')

Closed database successfully


In [6]:
# Drop non-beneficial identification columns
kushy_washington_df = kushy_washington_cleaned_df.drop(['test_id', 'test_strain'], axis=1)
print(kushy_washington_df.shape)
kushy_washington_df.head()

(722, 34)


,type,ailment_1,ailment_2,ailment_3,ailment_4,ailment_5,effects_1,effects_2,effects_3,effects_4,...,inventory_type,chemotype,strain_leafly_page_rank,strain_leafly_review_rank,strain_tests,strain_chemotype,strain_prop_chemotype1,strain_prop_chemotype2,strain_prop_chemotype3,strain_modal_chemo_prop
0,Sativa,Depression,Stress,Pain,Nausea,None,Happy,Energetic,Uplifted,Euphoric,...,Flower Lot,1,619,141,425,1,1.0,0.0,0.0,1.0
1,Hybrid,Stress,Depression,None,None,None,Uplifted,Euphoric,Creative,Relaxed,...,Flower Lot,1,1276,151,63,1,1.0,0.0,0.0,1.0
2,Sativa,Depression,Stress,None,None,None,Uplifted,Euphoric,Happy,Hungry,...,Flower Lot,1,0,0,0,0,0.0,0.0,0.0,0.0
3,Hybrid,Pain,Stress,Inflammation,Muscle Spasms,None,Relaxed,Creative,Sleepy,Happy,...,Flower Lot,1,1115,151,52,1,1.0,0.0,0.0,1.0
4,Hybrid,Stress,Depression,Pain,Muscle Spasms,None,Creative,Relaxed,Euphoric,Energetic,...,Flower Lot,1,708,141,124,1,1.0,0.0,0.0,1.0


In [7]:
# Determine number of unique values in each column
kushy_washington_df.nunique()

type                           4
ailment_1                      9
ailment_2                      9
ailment_3                     10
ailment_4                     10
ailment_5                      9
effects_1                     14
effects_2                     23
effects_3                     24
effects_4                     26
effects_5                     28
effects_6                      4
effects_7                      4
effects_8                      4
flavor_1                      32
flavor_2                      50
flavor_3                      42
ailment_count                  6
effects_count                  9
flavor_count                   4
thc_max                      697
cbd_max                      207
thc_cbd                      558
cbd_thc                       48
inventory_type                 7
chemotype                      3
strain_leafly_page_rank      640
strain_leafly_review_rank    128
strain_tests                 291
strain_chemotype               4
strain_pro

In [8]:
# Check datatypes
kushy_washington_df.dtypes

type                          object
ailment_1                     object
ailment_2                     object
ailment_3                     object
ailment_4                     object
ailment_5                     object
effects_1                     object
effects_2                     object
effects_3                     object
effects_4                     object
effects_5                     object
effects_6                     object
effects_7                     object
effects_8                     object
flavor_1                      object
flavor_2                      object
flavor_3                      object
ailment_count                  int64
effects_count                  int64
flavor_count                   int64
thc_max                      float64
cbd_max                      float64
thc_cbd                      float64
cbd_thc                      float64
inventory_type                object
chemotype                      int64
strain_leafly_page_rank        int64
s

In [9]:
# Use labelencoder to encode target ailment_1 column and check datatype
le = LabelEncoder()
kushy_washington_df['ailment_1'] = le.fit_transform(kushy_washington_df['ailment_1'])
kushy_washington_df['ailment_1']

0      0
1      8
2      0
3      7
4      8
      ..
717    7
718    1
719    8
720    6
721    6
Name: ailment_1, Length: 722, dtype: int32

In [10]:
# Check datatypes
kushy_washington_df.dtypes

type                          object
ailment_1                      int32
ailment_2                     object
ailment_3                     object
ailment_4                     object
ailment_5                     object
effects_1                     object
effects_2                     object
effects_3                     object
effects_4                     object
effects_5                     object
effects_6                     object
effects_7                     object
effects_8                     object
flavor_1                      object
flavor_2                      object
flavor_3                      object
ailment_count                  int64
effects_count                  int64
flavor_count                   int64
thc_max                      float64
cbd_max                      float64
thc_cbd                      float64
cbd_thc                      float64
inventory_type                object
chemotype                      int64
strain_leafly_page_rank        int64
s

In [11]:
# Convert vector of integers to one hot encoding using keras function to_categorical
dummy_y = np_utils.to_categorical(kushy_washington_df['ailment_1'])
print(dummy_y.shape)

(722, 9)


In [12]:
# Use get_dummies() to encode object columns
kushy_washington_df_encoded = pd.get_dummies(kushy_washington_df, columns=None)
print(kushy_washington_df_encoded.shape)
kushy_washington_df_encoded.head()

(722, 317)


,ailment_1,ailment_count,effects_count,flavor_count,thc_max,cbd_max,thc_cbd,cbd_thc,chemotype,strain_leafly_page_rank,...,flavor_3_Tree,flavor_3_Tropical,flavor_3_Woody,inventory_type_Bubble Hash,inventory_type_CO2 Hash Oil,inventory_type_Flower Lot,inventory_type_Food Grade Solvent Extract,inventory_type_Hash,inventory_type_Hydrocarbon Wax,inventory_type_Marijuana Mix
0,0,4,7,2,20.1309,0.54,37.28,0.03,1,619,...,0,0,0,0,0,1,0,0,0,0
1,8,2,7,2,9.9139,0.00,0.00,0.00,1,1276,...,0,0,0,0,0,1,0,0,0,0
2,0,2,7,1,10.8132,0.00,0.00,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0
3,7,4,7,2,11.6702,0.00,0.00,0.00,1,1115,...,0,0,0,0,0,1,0,0,0,0
4,8,4,8,2,13.4657,0.00,0.00,0.00,1,708,...,0,0,0,0,0,1,0,0,0,0


In [13]:
# Split preprocessed data into features and target arrays
X = kushy_washington_df_encoded.drop(['ailment_1'],axis=1).values
y = dummy_y
# Split preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=72)

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input/output features and hidden nodes for each layer
number_input_features = X_train.shape[1]
number_output_features = y.shape[1]
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 110
hidden_nodes_layer4 = 150
hidden_nodes_layer5 = 130
hidden_nodes_layer6 = 130

nn = tf.keras.models.Sequential()

# Input and first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Fifth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Sixth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=number_output_features, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 160)               50720     
                                                                 
 dense_1 (Dense)             (None, 80)                12880     
                                                                 
 dense_2 (Dense)             (None, 9)                 729       
                                                                 
Total params: 64,329
Trainable params: 64,329
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
batch_size = 17

# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=20*batch_size)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
17/17 [==============================] - 1s 3ms/step - loss: 1.7574 - accuracy: 0.4177
Epoch 2/100
17/17 [==============================] - 0s 2ms/step - loss: 0.7317 - accuracy: 0.8022
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 0.4238 - accuracy: 0.8909
Epoch 4/100
17/17 [==============================] - 0s 3ms/step - loss: 0.2678 - accuracy: 0.9519
Epoch 5/100
17/17 [==============================] - 0s 3ms/step - loss: 0.1703 - accuracy: 0.9797
Epoch 6/100
17/17 [==============================] - 0s 2ms/step - loss: 0.1131 - accuracy: 0.9945
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0775 - accuracy: 0.9982
Epoch 8/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0559 - accuracy: 1.0000
Epoch 9/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0420 - accuracy: 1.0000
Epoch 10/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0328 - accuracy: 1.0000
Epoch 11/

17/17 [==============================] - 0s 2ms/step - loss: 6.0537e-04 - accuracy: 1.0000
Epoch 79/100
17/17 [==============================] - 0s 2ms/step - loss: 5.9095e-04 - accuracy: 1.0000
Epoch 80/100
 1/17 [>.............................] - ETA: 0s - loss: 4.9492e-04 - accuracy: 1.0000
Epoch 80: saving model to checkpoints\weights.80.hdf5
17/17 [==============================] - 0s 5ms/step - loss: 5.7668e-04 - accuracy: 1.0000
Epoch 81/100
17/17 [==============================] - 0s 3ms/step - loss: 5.6361e-04 - accuracy: 1.0000
Epoch 82/100
17/17 [==============================] - 0s 3ms/step - loss: 5.5001e-04 - accuracy: 1.0000
Epoch 83/100
17/17 [==============================] - 0s 3ms/step - loss: 5.3796e-04 - accuracy: 1.0000
Epoch 84/100
17/17 [==============================] - 0s 3ms/step - loss: 5.2559e-04 - accuracy: 1.0000
Epoch 85/100
17/17 [==============================] - 0s 3ms/step - loss: 5.1348e-04 - accuracy: 1.0000
Epoch 86/100
17/17 [====================

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6/6 - 0s - loss: 1.0976 - accuracy: 0.7735 - 147ms/epoch - 25ms/step
Loss: 1.0976413488388062, Accuracy: 0.7734806537628174


In [81]:
# Export our model to HDF5 file
# nn.save("MMG_DLNN.h5")